# Домашняя работа 2

## Регрессия

С 1ого октября отменяется НДС на бриллианты, теперь они становятся новым инвестиционным инструментом. Давайте сделаем модель ценообразования для них.

Скачайте датасет diamonds.csv

В нем представлены характеристики бриллиантов и их цены с сайта  (B2C площадка) с 2022-07-01

**Описание полей**


* fluor - флуорисценуия (свойство камня светиться в лучах ультр)
* symmetry - показатель симметричности
* platform - название платформы, где был размещен камень
* quality_group - составной показатель из cut polish symmetry
* size_group - каратно весовая группа
* big_size_group - каратно-весовая группа
* shape - форма
* color - цвет
* clarity - чистота
* cut - качество огранки (может быть только у круглых камней)
* polish - полировка
* id - номер камня
* date - дата
* price - цена
* carat - кол-во карат
* tablepercent - размер площадки по отношению ширине
* price_per_carat - цена за карат
* z - длина (diameter)
* x - ширина
* depth_perc - отношение высоты к ширине
* y - высота



In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('diamonds.csv', index_col=0)

In [ ]:
df

,fluor,symmetry,platform,shape,color,clarity,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y,inflation_x,inflation_y,inflation
0,NONE,EX,jamesallen,PS,D,SI2,EX,14224443,202205,9120.0,1.51,5985.180270,10.67,6.23,59.0,3.67570,0.990967,1.0,1.0
1,NONE,VG,jamesallen,EM,K,SI2,VG,12902293,202205,1390.0,0.91,1379.221452,6.30,4.58,71.0,3.25180,0.902945,1.0,1.0
2,NONE,VG,jamesallen,PS,J,VS1,VG,12771762,202207,15990.0,2.27,7044.050000,11.66,7.26,62.2,4.51572,1.000000,1.0,1.0
3,MED,EX,jamesallen,RD,J,SI2,EX,14584509,202205,3700.0,1.01,3884.640107,6.37,6.42,62.0,3.98040,1.060401,1.0,1.0
4,NONE,EX,jamesallen,RD,G,SI1,EX,14773748,202206,7170.0,1.00,7533.370368,6.30,6.35,63.7,4.04495,1.050679,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155603,NONE,EX,jamesallen,RD,I,SI1,EX,15409696,202207,6040.0,1.01,5980.200000,6.46,6.49,60.8,3.94592,1.000000,1.0,1.0
155604,MED,EX,jamesallen,OV,H,VVS2,EX,15410303,202207,6620.0,1.01,6554.460000,7.76,5.40,65.0,3.51000,1.000000,1.0,1.0
155605,NONE,VG,jamesallen,EM,D,VS1,EX,15410668,202207,14750.0,1.50,9833.330000,7.84,5.43,65.8,3.57294,1.000000,1.0,1.0
155606,MED,EX,jamesallen,EM,H,SI2,EX,15410717,202207,48230.0,4.53,10646.800000,11.61,7.57,67.5,5.10975,1.000000,1.0,1.0


Необходимо предсказать `price_per_carat`

### Задание 1: Очистка

Не все камни успевают продаться за один месяц, поэтому в таблице есть повторы. Объедините данные по одному камню: подумайте, как лучше это сделать, какую цену брать.

**Попробуйте найти аномалии:** вдруг цена на некоторые камни сильно меняется (то есть продавец сам не знает, по какой цене их продавать). Также убедитесь, что остальные параметры камня не меняются.

In [ ]:
id_stat = df.groupby(by='id')['carat'].count().to_frame()
id_stat

,carat
id,
0,4
19423,3
19449,2
19476,1
19501,3
...,...
15418480,1
15418482,1
15418486,1


In [ ]:
df_one = df[df['id'].isin(id_stat[(id_stat['carat'] >= 2) & (id_stat['carat'] <= 3)].index)]  # те камни, которые встречались один раз
df_one

,fluor,symmetry,platform,shape,color,clarity,cut,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y
135269,NONE,EX,jamesallen,PS,F,SI1,NaN,EX,13870838,202206,12850.0,1.55,8290.32,10.29,6.41,62.0,3.97420
48477,MED,EX,jamesallen,RD,H,VVS2,EX,EX,11725253,202207,7510.0,1.02,7362.75,6.41,6.45,62.5,4.03125
236786,NONE,EX,jamesallen,EM,H,IF,NaN,EX,14444347,202205,21220.0,2.01,10557.21,8.60,6.37,65.0,4.14050
235781,NONE,EX,jamesallen,RD,E,VS2,EX,EX,14438434,202207,8660.0,1.00,8660.00,6.39,6.44,61.4,3.95416
277744,NONE,VG,jamesallen,RD,F,VS1,VG,EX,14615276,202206,8480.0,1.00,8480.00,6.28,6.36,62.3,3.96228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229133,NONE,EX,jamesallen,RD,I,SI2,EX,EX,14409793,202205,3330.0,0.90,3700.00,6.13,6.18,62.0,3.83160
268649,FNT,VG,jamesallen,RD,K,VS1,VG,EX,14581367,202205,3840.0,1.01,3801.98,6.27,6.33,63.0,3.98790
20595,NONE,EX,jamesallen,RD,D,VS2,EX,EX,7535786,202207,8700.0,0.97,8969.07,6.36,6.32,62.2,3.93104
282754,NONE,EX,jamesallen,RD,D,VS2,EX,EX,14631732,202206,6650.0,0.90,7388.89,6.13,6.18,61.7,3.81306


In [ ]:
id_stat = df.groupby(by='id')['carat'].count().to_frame() #Посчитайте статистику по кол-ву камней

# разделите выборку на две части

df_one = df[df['id'].isin(id_stat[id_stat['carat'] == 1].index)]  # те камни, которые встречались один раз
df_three = df[df['id'].isin(id_stat[(id_stat['carat'] >= 2) & (id_stat['carat'] <= 3)].index)] # те камни, которые встречались 2 или 3 раза

In [ ]:
stat = df_three.groupby('id')['price_per_carat', 'carat', 'z', 'x', 'depth_perc', 'y'].agg(np.std) # Посчитайте стандартное отклонение по нескольким полям,
#  подумайте где оно должно равняться 0, а где меняться в каких-то разумных пределах

<ipython-input-52-d98703aae4d4>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  stat = df_three.groupby('id')['price_per_carat', 'carat', 'z', 'x', 'depth_perc', 'y'].agg(np.std) # Посчитайте стандартное отклонение по нескольким полям,


Группировка происходит по уникальным идентификаторам ('id' камня), и стандартное отклонение будет рассчитываться для полей внутри каждого камня, то в данном случае ожидается, что стандартное отклонение будет ненулевым только для поля 'price_per_carat', так как другие поля имеют одинаковые значения (либо с очень маленькой погрешнестью) внутри одного камня.

In [ ]:
stat

,price_per_carat,carat,z,x,depth_perc,y
id,,,,,,
19423,318.916742,0.0,0.0,0.0,0.404145,0.026835
19449,423.054916,0.0,0.0,0.0,0.000000,0.000000
19501,415.692194,0.0,0.0,0.0,0.346410,0.027436
19504,0.000000,0.0,0.0,0.0,0.212132,0.014213
19505,404.564074,0.0,0.0,0.0,0.353553,0.028602
...,...,...,...,...,...,...
15409696,0.000000,0.0,0.0,0.0,0.000000,0.000000
15410303,0.000000,0.0,0.0,0.0,0.000000,0.000000
15410668,0.000000,0.0,0.0,0.0,0.000000,0.000000


In [ ]:
#В качество аномалий по цене решено было взять 5%
df_three['price_per_carat'].quantile(0.95)

14950.0

In [ ]:
df_three = df_three.query('price_per_carat<15000').sort_values(by=['id','date'])
df_three

,fluor,symmetry,platform,shape,color,clarity,cut,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y
5,NONE,EX,jamesallen,RD,D,VVS2,EX,EX,19423,202205,13970.0,1.05,13304.76,6.67,6.64,59.0,3.91760
6,NONE,EX,jamesallen,RD,D,VVS2,EX,EX,19423,202206,13970.0,1.05,13304.76,6.67,6.64,59.7,3.96408
7,NONE,EX,jamesallen,RD,D,VVS2,EX,EX,19423,202207,14550.0,1.05,13857.14,6.67,6.64,59.7,3.96408
8,NONE,EX,jamesallen,RD,D,VVS2,EX,EX,19449,202205,16850.0,1.17,14401.71,6.79,6.75,62.0,4.18500
22,NONE,VG,jamesallen,EM SQ,E,VS2,NaN,EX,19504,202205,23140.0,2.01,11512.44,7.03,6.70,73.0,4.89100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453529,NONE,VG,jamesallen,EM,D,VS1,NaN,EX,15410668,202207,14750.0,1.50,9833.33,7.84,5.43,65.8,3.57294
453543,MED,EX,jamesallen,EM,H,SI2,NaN,EX,15410717,202207,48230.0,4.53,10646.80,11.61,7.57,67.5,5.10975
453544,MED,EX,jamesallen,EM,H,SI2,NaN,EX,15410717,202207,48230.0,4.53,10646.80,11.61,7.57,67.5,5.10975
453566,NONE,EX,jamesallen,PR,G,VVS2,EX,EX,15412954,202207,7100.0,1.20,5916.67,5.80,5.76,72.4,4.17024


In [ ]:
df_three = df_three.groupby('id').last().reset_index()

In [ ]:
df_three

,id,fluor,symmetry,platform,shape,color,clarity,cut,polish,date,price,carat,price_per_carat,z,x,depth_perc,y
0,19423,NONE,EX,jamesallen,RD,D,VVS2,EX,EX,202207,14550.0,1.05,13857.14,6.67,6.64,59.7,3.96408
1,19449,NONE,EX,jamesallen,RD,D,VVS2,EX,EX,202205,16850.0,1.17,14401.71,6.79,6.75,62.0,4.18500
2,19504,NONE,VG,jamesallen,EM SQ,E,VS2,None,EX,202206,23140.0,2.01,11512.44,7.03,6.70,73.3,4.91110
3,19505,FNT,VG,jamesallen,RD,I,VVS1,EX,EX,202207,29130.0,2.01,14492.54,8.01,8.09,62.5,5.05625
4,19507,NONE,VG,jamesallen,EM SQ,E,VS1,None,VG,202207,27560.0,2.03,13576.35,7.20,6.86,70.1,4.80886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48004,15409696,NONE,EX,jamesallen,RD,I,SI1,EX,EX,202207,6040.0,1.01,5980.20,6.46,6.49,60.8,3.94592
48005,15410303,MED,EX,jamesallen,OV,H,VVS2,None,EX,202207,6620.0,1.01,6554.46,7.76,5.40,65.0,3.51000
48006,15410668,NONE,VG,jamesallen,EM,D,VS1,None,EX,202207,14750.0,1.50,9833.33,7.84,5.43,65.8,3.57294
48007,15410717,MED,EX,jamesallen,EM,H,SI2,None,EX,202207,48230.0,4.53,10646.80,11.61,7.57,67.5,5.10975


In [ ]:
df = pd.concat([df_one, df_three]).reset_index(drop=True) # соединяем результаты

In [ ]:
df

,fluor,symmetry,platform,shape,color,clarity,cut,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y
0,NONE,EX,jamesallen,PS,D,SI2,NaN,EX,14224443,202205,9120.0,1.51,6039.74,10.67,6.23,59.0,3.67570
1,NONE,VG,jamesallen,EM,K,SI2,NaN,VG,12902293,202205,1390.0,0.91,1527.47,6.30,4.58,71.0,3.25180
2,NONE,VG,jamesallen,PS,J,VS1,NaN,VG,12771762,202207,15990.0,2.27,7044.05,11.66,7.26,62.2,4.51572
3,MED,EX,jamesallen,RD,J,SI2,EX,EX,14584509,202205,3700.0,1.01,3663.37,6.37,6.42,62.0,3.98040
4,NONE,EX,jamesallen,RD,G,SI1,EX,EX,14773748,202206,7170.0,1.00,7170.00,6.30,6.35,63.7,4.04495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159621,NONE,EX,jamesallen,RD,I,SI1,EX,EX,15409696,202207,6040.0,1.01,5980.20,6.46,6.49,60.8,3.94592
159622,MED,EX,jamesallen,OV,H,VVS2,None,EX,15410303,202207,6620.0,1.01,6554.46,7.76,5.40,65.0,3.51000
159623,NONE,VG,jamesallen,EM,D,VS1,None,EX,15410668,202207,14750.0,1.50,9833.33,7.84,5.43,65.8,3.57294
159624,MED,EX,jamesallen,EM,H,SI2,None,EX,15410717,202207,48230.0,4.53,10646.80,11.61,7.57,67.5,5.10975


In [ ]:
df = df.drop('cut', axis=1)

Цены на бриллианты достаточно сильно меняются, попробуйте вычислить коэффициент инфляции и привести цены к последнему месяцу. Стоит учесть, что цены на все бриллианты не изменяются синхронно, то есть изменение в определенных группах может быть разным.

*Определите эти группы и рассчитайте коэффициенты инфляции для каждой из них. Подправьте цены на эти коэффициенты и создайте новую переменную.*

In [ ]:
categories = ['carat', 'color', 'clarity'] #определите по каким группам отпределять инфляцию (к примеру можно добавить каратную группу)
df_index = df.groupby(['date'] + categories)[['price_per_carat']]\
             .mean()\
             .reset_index() # индекс цен


# Включаем категориальные признаки в индекс, чтобы сопоставить с максимальной датой
df_index = df_index.merge(df_index.query('date == 202207')[categories + ['price_per_carat']]\
                          .rename(columns={'price_per_carat': 'price_per_carat_max'}),
                          on=categories, how='outer') # сопоставляем группы с максимальной датой

df_index['inflation'] = df_index.price_per_carat_max / df_index.price_per_carat # вычисляем инфляцию

df_with_inf = df.merge(df_index[['date'] + categories + ['inflation']],
                        on=['date'] + categories, how='left') # соединяем все в одной таблице


In [ ]:
df=df_with_inf.copy()
# Подправляем цены на основе коэффициентов инфляции
df['price_per_carat'] = df['price_per_carat'] * df['inflation']

In [ ]:
df_with_inf

,fluor,symmetry,platform,shape,color,clarity,polish,id,date,price,carat,price_per_carat,z,x,depth_perc,y,inflation
0,NONE,EX,jamesallen,PS,D,SI2,EX,14224443,202205,9120.0,1.51,6039.74,10.67,6.23,59.0,3.67570,0.990967
1,NONE,VG,jamesallen,EM,K,SI2,VG,12902293,202205,1390.0,0.91,1527.47,6.30,4.58,71.0,3.25180,0.902945
2,NONE,VG,jamesallen,PS,J,VS1,VG,12771762,202207,15990.0,2.27,7044.05,11.66,7.26,62.2,4.51572,1.000000
3,MED,EX,jamesallen,RD,J,SI2,EX,14584509,202205,3700.0,1.01,3663.37,6.37,6.42,62.0,3.98040,1.060401
4,NONE,EX,jamesallen,RD,G,SI1,EX,14773748,202206,7170.0,1.00,7170.00,6.30,6.35,63.7,4.04495,1.050679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159621,NONE,EX,jamesallen,RD,I,SI1,EX,15409696,202207,6040.0,1.01,5980.20,6.46,6.49,60.8,3.94592,1.000000
159622,MED,EX,jamesallen,OV,H,VVS2,EX,15410303,202207,6620.0,1.01,6554.46,7.76,5.40,65.0,3.51000,1.000000
159623,NONE,VG,jamesallen,EM,D,VS1,EX,15410668,202207,14750.0,1.50,9833.33,7.84,5.43,65.8,3.57294,1.000000
159624,MED,EX,jamesallen,EM,H,SI2,EX,15410717,202207,48230.0,4.53,10646.80,11.61,7.57,67.5,5.10975,1.000000


In [ ]:
df.isna().sum()

fluor                 7
symmetry              0
platform              0
shape                 0
color                 0
clarity               0
polish                0
id                    0
date                  0
price                 0
carat                 0
price_per_carat    4011
z                     0
x                     0
depth_perc            0
y                     0
inflation          4011
dtype: int64

In [ ]:

#DataFrame с группировкой по выбранным категориям и количеством уникальных групп
group_counts = df.groupby(categories).size().reset_index(name='group_count')

# Вывод результата
group_counts

,carat,color,clarity,group_count
0,0.81,D,FL,1
1,0.81,D,I1,3
2,0.81,D,IF,11
3,0.81,D,SI1,49
4,0.81,D,SI2,41
...,...,...,...,...
10983,12.01,F,VS2,1
10984,12.13,I,SI2,1
10985,15.03,J,SI2,1
10986,18.38,F,VVS2,1


### Задание 2: Модель

Определите функцию потерь (MSE или MAE) и аргументируйте выбор.
Попробуйте сделать baseline.
Используйте LableEncoder для категориальных фичей и **постройте линейную модель**.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression

df.dropna(inplace=True)

X = df.drop(columns=["price_per_carat"])
y = df["price_per_carat"]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13)

In [ ]:
# Кодирование категориальных признаков с помощью LabelEncoder
label_encoder = LabelEncoder()
X_encoded = X.copy()
for column in X_encoded.select_dtypes(include=['object']).columns:
    X_encoded[column] = label_encoder.fit_transform(X_encoded[column])

# Разделение данных на обучающую и тестовую выборки
X_train, X_valid, y_train, y_valid = train_test_split(X_encoded, y, test_size=0.2, random_state=13)

# Создание и обучение модели
model = LinearRegression()
model.fit(X_train, y_train)

# Прогнозирование на тестовой выборке
predictions = model.predict(X_valid)

# Вычисление MAE
mae = mean_absolute_error(y_valid, predictions)
print("MAE:", mae)

MAE: 1067.680201816351


MAE - это хорошая метрика для оценки модели в данном контексте, так как она измеряет среднюю абсолютную разницу между прогнозами и фактическими значениями. MAE позволяет оценить, насколько сильно модель ошибается в среднем по всем наблюдениям.

Теперь попробуйте OHE или TargetEncoder (сравните их).

Нормализуйте данные.

Поработайте с пропусками (обратите внимание на то, что у fluor возможен пропуск значения, а возможно отсутствие флуоресценции)

Покажите, насколько получилось улучшить результат.

In [ ]:
! pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 1.9 MB/s eta 0:00:00


In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder
from typing import Tuple, List


def OHE(df: pd.DataFrame, columns: List[str]) -> Tuple[pd.DataFrame, List[str]]:
    index = df.index
    one = OneHotEncoder(sparse=False, categories='auto')
    ohe = one.fit_transform(df[columns])
    col_names = one.get_feature_names_out(input_features=columns)
    df = df.drop(columns, axis=1)
    df = df.reset_index(drop=True)
    df = pd.concat([df, pd.DataFrame(ohe, columns=col_names)], axis=1)
    df = df.set_index(index)
    return df, col_names

def target_encode(df: pd.DataFrame, columns: List[str]) -> pd.DataFrame:
    encoder = TargetEncoder(cols=columns)
    df_encoded = encoder.fit_transform(df[columns], df['price_per_carat'])
    df.drop(columns=columns, inplace=True)
    df = pd.concat([df, df_encoded], axis=1)
    return df



categorical_columns = ['fluor', 'symmetry', 'platform', 'shape', 'color', 'clarity', 'polish']
df_ohe, ohe_col_names = OHE(df, categorical_columns)
df_encoded = target_encode(df, categorical_columns)
scaler = StandardScaler() # Помните, что на тесте делаем только transform
df_ohe_scaled = scaler.fit_transform(df_ohe)
df_encoded_scaled = scaler.fit_transform(df_encoded)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Сравните работу выбранной вами линейной модели с KNN

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

# Для линейной модели
X_train, X_valid, y_train, y_valid = train_test_split(df_ohe_scaled, y, test_size=0.2, random_state=13)
model = LinearRegression()
model.fit(X_train, y_train)
predictions_linear = model.predict(X_valid)
mae_linear = mean_absolute_error(y_valid, predictions_linear)

# Для KNN-регрессии
X_train, X_valid, y_train, y_valid = train_test_split(df_encoded_scaled, y, test_size=0.2, random_state=13)
model = KNeighborsRegressor()
model.fit(X_train, y_train)
predictions_knn = model.predict(X_valid)
mae_knn = mean_absolute_error(y_valid, predictions_knn)

print("MAE Linear Regression:", mae_linear)
print("MAE KNN Regression:", mae_knn)


MAE Linear Regression: 1.7873301042099912
MAE KNN Regression: 452.7597486717368


## Классификация

Загрузите датасет bodyPerformance

Описание:

https://www.kaggle.com/datasets/kukuroo3/body-performance-data

In [ ]:
df = pd.read_csv('bodyPerformance.csv') #укажите свой путь

In [ ]:
df

,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,class
0,27.0,M,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,C
1,25.0,M,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,A
2,31.0,M,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,C
3,32.0,M,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,B
4,28.0,M,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,B
...,...,...,...,...,...,...,...,...,...,...,...,...
13388,25.0,M,172.1,71.80,16.2,74.0,141.0,35.8,17.4,47.0,198.0,C
13389,21.0,M,179.7,63.90,12.1,74.0,128.0,33.0,1.1,48.0,167.0,D
13390,39.0,M,177.2,80.50,20.1,78.0,132.0,63.5,16.4,45.0,229.0,A
13391,64.0,F,146.1,57.70,40.4,68.0,121.0,19.3,9.2,0.0,75.0,D


### Задание 1: Определение гендера

Постройте модель, которая будет определять гендер.

Для этого определите две метрики качества, на которые вы будете смотреть. Аргументируйте свой выбор.

После чего преобразуйте категориальные переменные и постройте модель бинарной классификации на основе линейного классификатора и сравните с KNN.

P.S.: не забудте про нормализацию.

Две метрики будут использованы для оценки качества бинарных классификаторов:

1. Accuracy (Точность): Доля правильно классифицированных наблюдений от общего числа наблюдений. Она может быть хорошей метрикой, если классы в данных сбалансированы.

2. F1-Score: Это гармоническое среднее между точностью и полнотой (recall). F1-Score хорошо работает, когда классы несбалансированы.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score

# Копируем исходный датасет
df_copy = df.copy()


# Применяем One-Hot Encoding к категориальному признаку "gender"
ohe = OneHotEncoder(drop='first', sparse=False)
encoded_gender = ohe.fit_transform(df_copy[['gender']])
encoded_gender_df = pd.DataFrame(encoded_gender, columns=['gender_encoded'])
df_copy = pd.concat([df_copy, encoded_gender_df], axis=1)
df_copy.drop(columns=['gender'], inplace=True)
# Применяем One-Hot Encoding к категориальному признаку "class"
ohe = OneHotEncoder(drop='first', sparse=False)
encoded_class = ohe.fit_transform(df_copy[['class']])
encoded_class_df = pd.DataFrame(encoded_class, columns=ohe.get_feature_names_out(['class']))
df_copy = pd.concat([df_copy, encoded_class_df], axis=1)
df_copy.drop(columns=['class'], inplace=True)

# Разделяем данные на обучающий и валидационный наборы
X = df_copy.drop(columns=["gender_encoded"])
y = df_copy["gender_encoded"]
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13)





/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
# Нормализация данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

# Создание и обучение логистической регрессии
logreg_model = LogisticRegression()
logreg_model.fit(X_train_scaled, y_train)

# Создание и обучение KNN-классификатора
knn_model = KNeighborsClassifier()
knn_model.fit(X_train_scaled, y_train)

# Прогнозирование на валидационной выборке
logreg_predictions = logreg_model.predict(X_valid_scaled)
knn_predictions = knn_model.predict(X_valid_scaled)

# Вычисление метрик качества
logreg_accuracy = accuracy_score(y_valid, logreg_predictions)
logreg_f1 = f1_score(y_valid, logreg_predictions)

knn_accuracy = accuracy_score(y_valid, knn_predictions)
knn_f1 = f1_score(y_valid, knn_predictions)

print("Logistic Regression:")
print("Accuracy:", logreg_accuracy)
print("F1-Score:", logreg_f1)
print("")

print("K-Nearest Neighbors:")
print("Accuracy:", knn_accuracy)
print("F1-Score:", knn_f1)

Logistic Regression:
Accuracy: 0.9906681597611049
F1-Score: 0.992696465089103

K-Nearest Neighbors:
Accuracy: 0.9906681597611049
F1-Score: 0.9927049897869856


### Задание 2: Определение класса

Теперь опробуем построить модель, которая будет предсказывать class физической формы.

Для этого определите метрики качества для задачи мультиклассификации (аргументируйте выбор).

Постройте модель мультиклассовой классификации на основе линейного классификатора и сравните с KNN.

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder

# Копируем исходный датасет
df_copy = df.copy()

# Разделяем данные на обучающий и валидационный наборы
X = df_copy.drop(columns=["class"])
y = df_copy["class"]
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13)

# Создание кодировщика OneHotEncoder с параметром handle_unknown='ignore'
encoder = OneHotEncoder(drop="first", sparse=False, handle_unknown='ignore')

# Применение кодировщика к обучающей и валидационной выборкам
X_train_encoded = encoder.fit_transform(X_train)
X_valid_encoded = encoder.transform(X_valid)

# Создание и обучение модели линейного классификатора
linear_model = LogisticRegression(max_iter=1000)
linear_model.fit(X_train_encoded, y_train)
linear_predictions = linear_model.predict(X_valid_encoded)

# Создание и обучение модели KNN
knn_model = KNeighborsClassifier()
knn_model.fit(X_train_encoded, y_train)
knn_predictions = knn_model.predict(X_valid_encoded)

# Оценка метрик для линейной модели
print("Linear Model Metrics:")
print("Accuracy:", accuracy_score(y_valid, linear_predictions))
print("Precision:", precision_score(y_valid, linear_predictions, average='weighted'))
print("Recall:", recall_score(y_valid, linear_predictions, average='weighted'))
print("F1-Score:", f1_score(y_valid, linear_predictions, average='weighted'))
print(classification_report(y_valid, linear_predictions))

# Оценка метрик для KNN модели
print("\nKNN Model Metrics:")
print("Accuracy:", accuracy_score(y_valid, knn_predictions))
print("Precision:", precision_score(y_valid, knn_predictions, average='weighted'))
print("Recall:", recall_score(y_valid, knn_predictions, average='weighted'))
print("F1-Score:", f1_score(y_valid, knn_predictions, average='weighted'))
print(classification_report(y_valid, knn_predictions))


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2, 3, 4, 5, 6, 7, 8, 9, 10] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Linear Model Metrics:
Accuracy: 0.5244494214259052
Precision: 0.5238372841083675
Recall: 0.5244494214259052
F1-Score: 0.5236474389735247
              precision    recall  f1-score   support

           A       0.58      0.64      0.61       658
           B       0.40      0.38      0.39       690
           C       0.43      0.44      0.44       651
           D       0.68      0.64      0.66       680

    accuracy                           0.52      2679
   macro avg       0.52      0.53      0.52      2679
weighted avg       0.52      0.52      0.52      2679


KNN Model Metrics:
Accuracy: 0.3348264277715566
Precision: 0.3411287426099041
Recall: 0.3348264277715566
F1-Score: 0.33173096127603396
              precision    recall  f1-score   support

           A       0.39      0.47      0.43       658
           B       0.27      0.32      0.29       690
           C       0.31      0.30      0.30       651
           D       0.40      0.25      0.30       680

    accuracy        

Для задачи многоклассовой классификации, такой как предсказание класса физической формы, наиболее распространенными метриками являются:

1. Accuracy (Точность): Доля правильно классифицированных образцов среди всех образцов.

2. Precision (Точность для класса): Отношение верно классифицированных положительных образцов к общему числу предсказанных положительных образцов для данного класса. Это показатель того, насколько модель правильно классифицирует положительные случаи.

3. Recall (Полнота для класса): Отношение верно классифицированных положительных образцов к общему числу фактических положительных образцов для данного класса. Это показатель того, насколько модель обнаруживает все положительные случаи.

4. F1-Score: Гармоническое среднее между Precision и Recall. Это показатель, который объединяет оба аспекта качества модели.

5. Classification Report: Включает в себя Precision, Recall и F1-Score для каждого класса, а также средние значения для всех классов.

